In [8]:
import os
import numpy as np
from PIL import Image
from skimage.color import rgb2hsv
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
label_map = {
    'autumn': 0,
    'spring': 1,
    'summer': 2,
    'winter': 3
}


In [ ]:
X = []  
y = []  

for label_name, label_id in label_map.items():
    folder = os.path.join('faces_cleaned_only_augmentated', label_name)
    print(folder)
    if not os.path.isdir(folder):
        continue
    for filename in os.listdir(folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            new_path = os.path.join(folder, filename)
        try:
            image = Image.open(new_path).convert('RGB')
            image_np = np.array(image.resize((128, 128))) / 255.0 

            mean_rgb = image_np.mean(axis=(0, 1))  # mean rgb
            hsv = rgb2hsv(image_np)                # convert to HSV
            mean_hsv = hsv.mean(axis=(0, 1))       # mean hsv 
            std_rgb = image_np.std(axis=(0, 1))    # std rgb
            std_hsv = hsv.std(axis=(0, 1))         # std hsv
            
            features = np.concatenate([mean_rgb, std_rgb, mean_hsv, std_hsv])
            
            h_hist, _ = np.histogram(hsv[:, :, 0], bins=8, range=(0, 1), density=True)
            s_hist, _ = np.histogram(hsv[:, :, 1], bins=8, range=(0, 1), density=True)
            v_hist, _ = np.histogram(hsv[:, :, 2], bins=8, range=(0, 1), density=True)

            features = np.concatenate([features, h_hist, s_hist, v_hist])

            X.append(features)
            y.append(label_id)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

faces_cleaned_only_augmentated\autumn
faces_cleaned_only_augmentated\spring
faces_cleaned_only_augmentated\summer
faces_cleaned_only_augmentated\winter


In [8]:
import csv

with open('colortype_dataset.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for xi, yi in zip(X, y):
        writer.writerow(xi + [yi])

In [9]:
X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
model = RandomForestClassifier(n_estimators=300,random_state=42)

model.fit(X_train, y_train)

,n_estimators,300
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [11]:
y_pred = model.predict(X_val) 

print("Accuracy:", accuracy_score(y_val, y_pred))
print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=label_map.keys()))


Accuracy: 0.8537200504413619
Classification Report:
              precision    recall  f1-score   support

      autumn       0.85      0.85      0.85       270
      spring       0.92      0.75      0.83       113
      summer       0.94      0.73      0.82       100
      winter       0.82      0.93      0.87       310

    accuracy                           0.85       793
   macro avg       0.88      0.82      0.84       793
weighted avg       0.86      0.85      0.85       793



In [13]:
import joblib
joblib.dump(model, 'random_forest_2.pkl')

['random_forest_2.pkl']

In [ ]:
import joblib

model = joblib.load('random_forest.pkl')

In [10]:
def take_photos(path):
    X = []  
    y = []  

    for label_name, label_id in label_map.items():
        folder = os.path.join(path, label_name)
        print(folder)
        if not os.path.isdir(folder):
            continue
        for filename in os.listdir(folder):
            if filename.lower().endswith((".jpg", ".jpeg", ".png")):
                new_path = os.path.join(folder, filename)
            try:
                image = Image.open(new_path).convert('RGB')
                image_np = np.array(image.resize((128, 128))) / 255.0 

                mean_rgb = image_np.mean(axis=(0, 1))  # mean rgb
                hsv = rgb2hsv(image_np)                # convert to HSV
                mean_hsv = hsv.mean(axis=(0, 1))       # mean hsv 
                std_rgb = image_np.std(axis=(0, 1))    # std rgb
                std_hsv = hsv.std(axis=(0, 1))         # std hsv
                
                features = np.concatenate([mean_rgb, std_rgb, mean_hsv, std_hsv])
                
                h_hist, _ = np.histogram(hsv[:, :, 0], bins=8, range=(0, 1), density=True)
                s_hist, _ = np.histogram(hsv[:, :, 1], bins=8, range=(0, 1), density=True)
                v_hist, _ = np.histogram(hsv[:, :, 2], bins=8, range=(0, 1), density=True)

                features = np.concatenate([features, h_hist, s_hist, v_hist])

                X.append(features)
                y.append(label_id)

            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return X, y

In [11]:
X, y = take_photos('DATASET')

DATASET\autumn
DATASET\spring
DATASET\summer
DATASET\winter


In [12]:
y_pred = model.predict(X)
print("Accuracy:", accuracy_score(y, y_pred))
print("Classification Report:")
print(classification_report(y, y_pred, target_names=label_map.keys()))

Accuracy: 0.553921568627451
Classification Report:
              precision    recall  f1-score   support

      autumn       0.54      0.56      0.55        50
      spring       0.64      0.35      0.45        52
      summer       0.53      0.37      0.44        51
      winter       0.55      0.94      0.69        51

    accuracy                           0.55       204
   macro avg       0.56      0.55      0.53       204
weighted avg       0.56      0.55      0.53       204



In [13]:
X, y = take_photos('colortypes')

colortypes\autumn
colortypes\spring
colortypes\summer
colortypes\winter


In [14]:
y_pred = model.predict(X)
print("Accuracy:", accuracy_score(y, y_pred))
print("Classification Report:")
print(classification_report(y, y_pred, target_names=label_map.keys()))

Accuracy: 0.5161290322580645
Classification Report:
              precision    recall  f1-score   support

      autumn       0.39      0.55      0.46        67
      spring       0.65      0.41      0.50        63
      summer       0.73      0.28      0.41        39
      winter       0.55      0.68      0.61        79

    accuracy                           0.52       248
   macro avg       0.58      0.48      0.49       248
weighted avg       0.56      0.52      0.51       248

